# Data Preparation

## Table of Contents

1. [Data Cleaning for Baseline Model](#dc-base)
    - [1.1 Marital Status One Hot Encoding](#step1)
    - [1.2 Application Mode One Hot Encoding](#step2)
    - [1.3 Course One Hot Encoding](#step3)
    - [1.4 Rename Daytime/Nighttime Attendance](#step4)
    - [1.5 Modify Previous Qualifications](#step5)
    - [1.6 Nationality Mapping](#step6)
    - [1.7 Modify Mother's Qualifications](#step7)
    - [1.8 Modify Father's Qualifications](#step8)
    - [1.9 Mother's Occupation Cleaning](#step9)
    - [1.10 Father's Occupation Cleaning](#step10)
    - [1.11 Rename Gender Column](#step11)
    - [1.12 Encode the Target](#step12)

In [455]:
import pandas as pd

data = pd.read_csv("data/student-dropout-academic-success.csv", sep=";")
df = data.copy()
data.head(3)

,Marital status,Application mode,Application order,Course,Daytime/evening attendance\t,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,1,17,5,171,1,1,122.0,1,19,12,...,0,0,0,0,0.000000,0,10.8,1.4,1.74,Dropout
1,1,15,1,9254,1,1,160.0,1,1,3,...,0,6,6,6,13.666667,0,13.9,-0.3,0.79,Graduate
2,1,1,5,9070,1,1,122.0,1,37,37,...,0,6,0,0,0.000000,0,10.8,1.4,1.74,Dropout


In [456]:
df.columns = df.columns.str.lower().str.replace(" ", "_").str.strip()
# df.columns

## Data Cleaning for Baseline Model <a class="anchor" id="dc-base"></a>

We will start by cleaning the data for the baseline model. This will be for a target variable as is with Dropout, Enrolled, and Graduated.

### 1.1 Marital Status <a class="anchor" id="step1"></a>

For now, we will convert this to a One Hot Encoded variable.

In [457]:
df.marital_status.unique()

array([1, 2, 4, 3, 5, 6], dtype=int64)

In [458]:
# mapping to string values so one hot encoding columns make sense
df.marital_status = df.marital_status.map(
    {1: "single", 2: "married", 3: "widower", 4: "divorced", 5: "facto_union", 6: "legally_separated"}
)

In [459]:
assert df.marital_status.isna().sum() == 0

In [460]:
marital_status_df = pd.get_dummies(df.marital_status, "marital_status", dtype="int") \
    .drop("marital_status_single", axis=1)
df = df.join(marital_status_df)
df = df.drop(columns="marital_status")
df.shape

(4424, 41)

### 1.2 Application Mode <a class="anchor" id="step2"></a>

Need to convert back to labels then One Hot Encode them. For now I will just drop first because it is hard to get a column to drop through intuition.

In [461]:
df.application_mode = df.application_mode.map({
    1: "1st_phase_general_contingent", 2: "Ordinance_No_612/93",
    5: "1st_phase_special_contingent_Azores_Island",
    7: "Holders_of_other_higher_courses", 10: "Ordinance_No_854-B/99",
    15: "International_student_bachelor",
    16: "1st_phase_special_contingent_Madeira_Island",
    17: "2nd_phase_general_contingent", 18: "3rd_phase_general_contingent",
    26: "Ordinance_No_533-A/99_item_b2_Different_Plan",
    27: "Ordinance_No_533-A/99_item_b3_Other_Institution",
    39: "Over_23_years_old", 42: "Transfer", 43: "Change_of_course",
    44: "Technological_specialization_diploma_holders",
    51: "Change_of_institution_course", 53: "Short_cycle_diploma_holders",
    57: "Change_of_institution_course_International"
})

In [462]:
assert df.application_mode.isna().sum() == 0

In [463]:
application_mode_dummies = pd.get_dummies(df.application_mode, "application_mode", drop_first=True, dtype="int")
# application_mode_dummies

In [464]:
df = df.join(application_mode_dummies)
df = df.drop(columns="application_mode")
df.shape

(4424, 57)

### 1.3 Course <a class="anchor" id="step3"></a>

For this feature I need to map integers to real values then One Hot Encode them into categories.

I will use Nursing as the reference column due to the fact it has the most rows.

In [465]:
# mapping course numbers to their respective name
df.course = df.course.map({
    33: "Biofuel_Production_Technologies", 171: "Animation_and_Multimedia_Design",
    8014: "Social_Service_evening_attendance", 9003: "Agronomy",
    9070: "Communication_Design", 9085: "Veterinary_Nursing",
    9119: "Informatics_Engineering", 9130: "Equinculture",
    9147: "Management", 9238: "Social_Service",
    9254: "Tourism", 9500: "Nursing",
    9556: "Oral_Hygiene", 9670: "Advertising_and_Marketing_Management",
    9773: "Journalism_and_Communication", 9853: "Basic_Education",
    9991: "Management_evening_attendance"
})

In [466]:
assert df.course.isna().sum() == 0

In [467]:
courses_dummies = pd.get_dummies(df.course, "course", dtype="int") \
    .drop(columns=["course_Nursing"])
# courses_dummies.columns

In [468]:
df = df.join(courses_dummies)
df = df.drop(columns="course")
df.shape

(4424, 72)

### 1.4 Daytime Nightime Attendance <a class="anchor" id="step4"></a>

In [469]:
# rename column for clarity
df = df.rename(columns={"daytime/evening_attendance": "daytime_attendance"})

### 1.5 Previous Qualifications <a class="anchor" id="step5"></a>

For now, I will encode this feature as an Ordinal feature

In [470]:
mapper = {
    1: "high_school", 2: "higher_education", 3: "higher_education", 4: "graduate_school",
    5: "graduate_school", 6: "higher_education", 9: "other", 10: "other", 11: "other", 12: "other",
    14: "other", 15: "other", 18: "technical_training", 19: "other", 22: "technical_training", 26: "other",
    27: "other", 29: "other", 30: "other", 34: "other", 35: "other", 36: "other", 37: "other",
    38: "other", 39: "technical_training",
    40: "higher_education", 41: "technical_training", 42: "technical_training", 43: "graduate_school",
    44: "graduate_school"
}

In [471]:
df.previous_qualification = df.previous_qualification.map(mapper)
assert df.previous_qualification.isna().sum() == 0
df.previous_qualification.value_counts()

previous_qualification
high_school           3717
technical_training     255
other                  232
higher_education       205
graduate_school         15
Name: count, dtype: int64

In [472]:
from sklearn.preprocessing import OrdinalEncoder

ord_encoder = OrdinalEncoder(
    categories=[["other", "high_school", "technical_training", "higher_education", "graduate_school"]]
)
ord_encoder.set_output(transform="pandas")
df.previous_qualification = ord_encoder.fit_transform(df[["previous_qualification"]])

In [473]:
df.previous_qualification.value_counts()

previous_qualification
1.0    3717
2.0     255
0.0     232
3.0     205
4.0      15
Name: count, dtype: int64

### 1.6 Nationality <a class="anchor" id="step6">

Just mapping these to One Hot encoded columns

In [474]:
df = df.rename(columns={"nacionality": "nationality"})  # renaming column

# mapping column to categories
df.nationality = df.nationality.map({
    1: "Portuguese", 2: "German", 6: "Spanish", 11: "Italian",
    13: "Dutch", 14: "English", 17: "Lithuanian", 21: "Angolan",
    22: "Cape_Verdean", 24: "Guinean", 25: "Mozambican",
    26: "Santomean", 32: "Turkish", 41: "Brazilian",
    62: "Romanian", 100: "Moldova_Republic_of",
    101: "Mexican", 103: "Ukrainian", 105: "Russian",
    108: "Cuban", 109: "Colombian"
})

In [475]:
assert df.nationality.isna().sum() == 0

In [476]:
nationality_to_dummies = pd.get_dummies(df.nationality, "nationality", dtype="int") \
    .drop(columns=["nationality_Portuguese"])
df = df.join(nationality_to_dummies)
df = df.drop(columns="nationality")
df.shape

(4424, 91)

### 1.7 Mother's Qualification <a class="anchor" id="step7"></a>

This feature was similar enough to Previous Qualifications to be able to use prior mapper.

In [477]:
df["mother's_qualification"] = df["mother's_qualification"].map(mapper)
assert df["mother's_qualification"].isna().sum() == 0
ord_encoder2 = OrdinalEncoder(
    categories=[["other", "high_school", "technical_training", "higher_education", "graduate_school"]]
)
ord_encoder2.set_output(transform="pandas")
df["mother's_qualification"] = ord_encoder2.fit_transform(df[["mother's_qualification"]])

### 1.8 Father's Qualification <a class="anchor" id="step8"></a>

Will have to construct a new mapper, then encode feature as a Ordinal Feature

In [478]:
mapper_father = {}
other_keys = [7, 8, 9, 10, 11, 12, 17, 18, 19, 20, 21, 24, 25, 26, 27, 28, 35, 36, 37, 38]
high_school_keys = [1, 14]
technical_training_keys = [13, 15, 16, 22, 23, 29, 32, 39, 41, 42]
higher_ed_keys = [2, 3, 6, 30, 31, 40]
graduate_school_keys = [4, 5, 33, 34, 43, 44]

In [479]:
def add_to_dict(mapper_dict, key_list_, value):
    for key in key_list_:
        mapper_dict[key] = value
    return mapper_dict

In [480]:
in_data = set(df["father's_qualification"].unique())

In [481]:
categories = ["other", "high_school", "technical_training", "higher_education", "graduate_school"]
key_lists = [other_keys, high_school_keys, technical_training_keys, higher_ed_keys, graduate_school_keys]
for category, key_list in zip(categories, key_lists):
    mapper_father = add_to_dict(mapper_father, key_list, category)

In [482]:
in_mapper = set(mapper_father.keys())
in_data.difference(in_mapper)

set()

In [483]:
df["father's_qualification"] = df["father's_qualification"].map(mapper_father)
assert df["father's_qualification"].isna().sum() == 0

In [484]:
ord_encoder3 = OrdinalEncoder(
    categories=[["other", "high_school", "technical_training", "higher_education", "graduate_school"]]
)
ord_encoder3.set_output(transform="pandas")
df["father's_qualification"] = ord_encoder3.fit_transform(df[["father's_qualification"]])

### 1.9 Mother's Occupation <a class="anchor" id="step9"></a>

In [485]:
mother_job_mapper = {
    0: "Student",
    1: "Representatives of the Legislative Power and Executive Bodies, Directors, Directors and Executive Managers",
    2: "Specialists in Intellectual and Scientific Activities",
    3: "Intermediate Level Technicians and Professions",
    4: "Administrative staff",
    5: "Personal Services, Security and Safety Workers and Sellers",
    6: "Farmers and Skilled Workers in Agriculture, Fisheries and Forestry",
    7: "Skilled Workers in Industry, Construction and Craftsmen",
    8: "Installation and Machine Operators and Assembly Workers",
    9: "Unskilled Workers",
    10: "Armed Forces Professions",
    90: "Other Situation",
    99: "(blank)",
    122: "Health professionals",
    123: "Teachers",
    125: "Specialists in information and communication technologies (ICT)",
    131: "Intermediate level science and engineering technicians and professions",
    132: "Technicians and professionals, of intermediate level of health",
    134: "Intermediate level technicians from legal, social, sports, cultural and similar services",
    141: "Office workers, secretaries in general and data processing operators",
    143: "Data, accounting, statistical, financial services and registry-related operators",
    144: "Other administrative support staff",
    151: "Personal service workers",
    152: "Sellers",
    153: "Personal care workers and the like",
    171: "Skilled construction workers and the like, except electricians",
    173: "Skilled workers in printing, precision instrument manufacturing, jewelers, artisans and the like",
    175: "Workers in food processing, woodworking, clothing and other industries and crafts",
    191: "Cleaning workers",
    192: "Unskilled workers in agriculture, animal production, fisheries and forestry",
    193: "Unskilled workers in extractive industry, construction, manufacturing and transport",
    194: "Meal preparation assistants"
}

In [486]:
assert df["mother's_occupation"].map(mother_job_mapper).isna().sum() == 0

In [487]:
df["mother's_occupation"] = df["mother's_occupation"].map(mother_job_mapper)
df["mother's_occupation"].value_counts()

mother's_occupation
Unskilled Workers                                                                                             1577
Administrative staff                                                                                           817
Personal Services, Security and Safety Workers and Sellers                                                     530
Intermediate Level Technicians and Professions                                                                 351
Specialists in Intellectual and Scientific Activities                                                          318
Skilled Workers in Industry, Construction and Craftsmen                                                        272
Student                                                                                                        144
Representatives of the Legislative Power and Executive Bodies, Directors, Directors and Executive Managers     102
Farmers and Skilled Workers in Agriculture, Fisheries and Fo

In [488]:
mother_job_dummies = pd.get_dummies(df["mother's_occupation"], "mother_job", dtype="int") \
    .drop(columns=["mother_job_Unskilled Workers"])

In [489]:
df = df.join(mother_job_dummies)
df = df.drop(columns="mother's_occupation")
assert df.shape[0] == 4424

### 1.10 Father's Occupation <a class="anchor" id="step10"></a>

In [490]:
father_job_mapper = {
    0: "Student",
    1: "Representatives of the Legislative Power and Executive Bodies, Directors, Directors and Executive Managers",
    2: "Specialists in Intellectual and Scientific Activities",
    3: "Intermediate Level Technicians and Professions",
    4: "Administrative staff",
    5: "Personal Services, Security and Safety Workers and Sellers",
    6: "Farmers and Skilled Workers in Agriculture, Fisheries and Forestry",
    7: "Skilled Workers in Industry, Construction and Craftsmen",
    8: "Installation and Machine Operators and Assembly Workers",
    9: "Unskilled Workers",
    10: "Armed Forces Professions",
    90: "Other Situation",
    99: "(blank)",
    101: "Armed Forces Officers",
    102: "Armed Forces Sergeants",
    103: "Other Armed Forces personnel",
    112: "Directors of administrative and commercial services",
    114: "Hotel, catering, trade and other services directors",
    121: "Specialists in the physical sciences, mathematics, engineering and related techniques",
    122: "Health professionals",
    123: "Teachers",
    124: "Specialists in finance, accounting, administrative organization, public and commercial relations",
    131: "Intermediate level science and engineering technicians and professions",
    132: "Technicians and professionals, of intermediate level of health",
    134: "Intermediate level technicians from legal, social, sports, cultural and similar services",
    135: "Information and communication technology technicians",
    141: "Office workers, secretaries in general and data processing operators",
    143: "Data, accounting, statistical, financial services and registry-related operators",
    144: "Other administrative support staff",
    151: "Personal service workers",
    152: "Sellers",
    153: "Personal care workers and the like",
    154: "Protection and security services personnel",
    161: "Market-oriented farmers and skilled agricultural and animal production workers",
    163: "Farmers, livestock keepers, fishermen, hunters and gatherers, subsistence",
    171: "Skilled construction workers and the like, except electricians",
    172: "Skilled workers in metallurgy, metalworking and similar",
    174: "Skilled workers in electricity and electronics",
    175: "Workers in food processing, woodworking, clothing and other industries and crafts",
    181: "Fixed plant and machine operators",
    182: "Assembly workers",
    183: "Vehicle drivers and mobile equipment operators",
    192: "Unskilled workers in agriculture, animal production, fisheries and forestry",
    193: "Unskilled workers in extractive industry, construction, manufacturing and transport",
    194: "Meal preparation assistants",
    195: "Street vendors (except food) and street service providers"
}

In [491]:
assert (mapped_column_father:=df["father's_occupation"].map(father_job_mapper)).isna().sum() == 0
assert mapped_column_father.size == 4424

In [492]:
mapped_column_father.value_counts().sort_values(ascending=False).iloc[:5]

father's_occupation
Unskilled Workers                                             1010
Skilled Workers in Industry, Construction and Craftsmen        666
Personal Services, Security and Safety Workers and Sellers     516
Administrative staff                                           386
Intermediate Level Technicians and Professions                 384
Name: count, dtype: int64

In [493]:
father_job_dummies = pd.get_dummies(mapped_column_father, "father_job", dtype="int")\
    .drop(columns=["father_job_Unskilled Workers"])

In [494]:
assert father_job_dummies.shape == (4424, len(df["father's_occupation"].unique()) - 1)

In [495]:
df = df.join(father_job_dummies)
df = df.drop(columns="father's_occupation")
assert df.shape[0] == 4424

### 1.11 Gender Rename <a class="anchor" id="step11"></a>

In [496]:
df = df.rename({"gender": "is_male"})

In [497]:
df.shape

(4424, 165)

### 1.12 Encode the Target <a class="anchor" id="step12"></a>

In [498]:
target_map = {"Enrolled": 1, "Dropout": 0, "Graduate": 2}
df["target"] = df["target"].map(target_map)

### Write to file

In [499]:
df.select_dtypes(exclude="object").dtypes.value_counts()

int32      134
int64       21
float64     10
Name: count, dtype: int64

In [500]:
df = df.select_dtypes(exclude="object")
df.to_csv("data/cleaned_data_baseline.csv", index=False)

## Feature Engineering

This portion of notebook is where we modify dataset to gain value from best features.

In [501]:
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler

In [502]:
def score_model(x, y, metric="accuracy"):
    x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)
    scaler = MinMaxScaler()
    forest = RandomForestClassifier(random_state=42, class_weight={0:7, 1:3})
    standardized_forest = make_pipeline(scaler, forest)
    return cross_val_score(standardized_forest, x_train, y_train, scoring=metric)

In [503]:
def run_model(data_: pd.DataFrame, metric="accuracy"):
    # split data
    fn_data = data_.copy()
    y = fn_data.pop("target")
    x = fn_data
    
    # train model
    return score_model(x, y, metric)

In [504]:
df.target = df.target.map(lambda x: 1 if x == 0 else 0)

In [505]:
# test with our baseline processed data
print(f"Average Validation Accuracy: {run_model(df).mean():.2f}")
print(f"Average Validation Recall: {run_model(df, "recall").mean():.2f}")

Average Validation Accuracy: 0.88
Average Validation Recall: 0.71


In [506]:
fe_df = pd.read_csv("data/student-dropout-academic-success.csv", sep=";")
fe_df.columns = fe_df.columns.str.lower().str.replace(" ", "_")
fe_df["target"] = fe_df["target"].map(lambda x: 1 if x == "Dropout" else 0)
fe_df.head()

,marital_status,application_mode,application_order,course,daytime/evening_attendance\t,previous_qualification,previous_qualification_(grade),nacionality,mother's_qualification,father's_qualification,...,curricular_units_2nd_sem_(credited),curricular_units_2nd_sem_(enrolled),curricular_units_2nd_sem_(evaluations),curricular_units_2nd_sem_(approved),curricular_units_2nd_sem_(grade),curricular_units_2nd_sem_(without_evaluations),unemployment_rate,inflation_rate,gdp,target
0,1,17,5,171,1,1,122.0,1,19,12,...,0,0,0,0,0.000000,0,10.8,1.4,1.74,1
1,1,15,1,9254,1,1,160.0,1,1,3,...,0,6,6,6,13.666667,0,13.9,-0.3,0.79,0
2,1,1,5,9070,1,1,122.0,1,37,37,...,0,6,0,0,0.000000,0,10.8,1.4,1.74,1
3,1,17,2,9773,1,1,122.0,1,38,37,...,0,6,10,5,12.400000,0,9.4,-0.8,-3.12,0
4,2,39,1,8014,0,1,100.0,1,37,38,...,0,6,6,6,13.000000,0,13.9,-0.3,0.79,0


### 2.1 Marital Status


In [507]:
fe_df.marital_status = fe_df.marital_status.map({
    1: "single", 2: "married", 3: "widower", 4: "divorced", 
    5: "facto_union", 6: "legally_separated"
})
fe_df["marital_status"].value_counts()

marital_status
single               3919
married               379
divorced               91
facto_union            25
legally_separated       6
widower                 4
Name: count, dtype: int64

In [508]:
marital_status = pd.get_dummies(fe_df["marital_status"], dtype=int)\
    .drop(columns="single")
fe_df = fe_df.join(marital_status).drop(columns="marital_status")
fe_df.shape

(4424, 41)